In [1]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import imutils

from src.handDetector import HandDetector

objc[92854]: Class CaptureDelegate is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x13d476480) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x12dd6c860). One of the two will be used. Which one is undefined.
objc[92854]: Class CVWindow is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x13d4764d0) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11fc94a68). One of the two will be used. Which one is undefined.
objc[92854]: Class CVView is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x13d4764f8) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.

4.6.0


In [2]:
data_file_out = "out/lala.csv"

def get_init_gestures():
    gestures = ['ok', 'victory', 'call', 'hang_in', 'one_up', 'two_up', 'hand_closed', 'hand_open', 'machedici', 'random']
    gestures = {ord('a')+x: [gestures[x], 0] for x in range(len(gestures))}
    return gestures

In [3]:

def put_text(img, text):

  font = cv2.FONT_HERSHEY_SIMPLEX
  fontScale = 0.5
  color = (255, 0, 0)
  thickness=1

  y0, dy = 15, 15
  for i, line in enumerate(text):
    y = y0 + i*dy
    cv2.putText(img, line, (5, y ), font, fontScale, color, thickness, cv2.LINE_AA)


def warning(img):
  img = cv2.circle(img, (int(img.shape[1]/2),int(img.shape[0]/2)), 20, (0, 0, 255), -1)

  return img

## Record Gestures

In [4]:
detector = HandDetector()
cap = cv2.VideoCapture(0)

SHOW_WIDTH = 800
NO_KEY = -1
ESC_KEY = 27
WAIT_KEY = 100

gestures = get_init_gestures()
results = []

while cap.isOpened():

  success, img = cap.read()
  if not success:
    break

  k = cv2.waitKey(WAIT_KEY)
  if k == ESC_KEY:
    break
  
  img = imutils.resize(img, width=SHOW_WIDTH)
  detector.fit(img)

  # There is a hand on the screen
  if detector.found_hands():

    if k != NO_KEY:

      lm_norms = detector.get_positions_normalized()

      for h_k, lm_norm in enumerate(lm_norms):
        # mediapipe tries to guess hands parts if they are not in the 
        # screen, let's not take those
        if detector.hand_fits_screen(h_k):
          try:
            gestures[k][1] += 1
            results.append([gestures[k][1] , lm_norm])
          except KeyError:
            img = warning(img)     

        else: 
          img = warning(img)        

    # Main window visuals
    img = detector.transform_draw(img)
    img = detector.transform_connect_lines(img)
  
  text = [f"[{chr(x[0])}]-{x[1][0]}: {x[1][1]}: " for x in gestures.items()]
  put_text(img, text)
  cv2.imshow("Image", img)


cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Save in file

In [ ]:
target = [r[0] for r in results]
array = [r[1] for r in results]


df = pd.DataFrame(array)
df = df.round(4)
cols = df.columns
df['target'] = target

# Target should be at the beginning
cols = cols.insert(0, 'target')

df[cols].to_csv("out/lala.csv", index=False, header=False, mode='a')

In [46]:
a = {1: ["bar",34], 2:[ "lolo",0], 3: ["lala", 3]}


a[34]


KeyError: 34

In [32]:
for n in a.items():
    print (n[1][0])

bar
lolo
lala
